In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

In [35]:
onerpmdata = pd.read_excel(r"Data File.xlsx", "Master Sheet")
exercisedata = pd.read_excel(r"Data File.xlsx", "Exercise Routines")

In [28]:
class fitnessanalytics:

    #Creating a fucntion that gets us max 1rpm for given date range
    def OneRPM(self, df, dategreaterthan = '1990-01-01', currentlycutting = 0): #classes me first argument must always be self
        datefilter = df["Date"] >= dategreaterthan #setting dates greater than input date
        df = df[datefilter]
        exercisesonlyfilter = (df["Category"] == "Exercise") #filtering for exercises only under category to shorten and speed up dataset
        df = df[exercisesonlyfilter]
        df["1rpms (KG)"] = (((df["Repetitions performed"] / 30) + 1) * df["Weight (In Kg)"]) #calculating 1rpms in (KG)
        
        if currentlycutting == 1:
            df["1rpms (KG)"] =  df["1rpms (KG)"] * 1 #putting an if else as last me cut dalenge to all rpms will be reduced by 30% as we will be fasted! anything else dala to just pass!
        else:
            pass
        
        df = df.groupby(["Name of the exercise"], sort = False)["1rpms (KG)"].max() #applying group by to get max 1RPM for given date range per exercise
        df = pd.DataFrame(df) #after groupby, it wont be a dataframe anymore so converting back to a dataframe
        df["1rpms (LB)"] = df["1rpms (KG)"]*2.2 #calculating 1rpms (in LBS)
        df[["1rpms (KG)", "1rpms (LB)"]] = round(df[["1rpms (KG)", "1rpms (LB)"]], 2) #rounding to 2 decimal places for better interpretation
        self.df = df
    
    def workout(self, exercisedata, week = 1, dayoftheweek = 1, randomreps = 0, minreps = 6, maxreps= 12):
        onerpmdata = self.df #calling the onerpm fucntion from the class
        exercisedata = exercisedata #assigning workout dataframe here
        weekfilter = exercisedata["Week"] == week #filtering for weeks this should dieally be phase!..
        exercisedata = exercisedata[weekfilter] #appluying the week filter
        dayfilter = exercisedata["Day"] == dayoftheweek #filtering for day 1,2,3,4,5 of the workout..
        exercisedata = exercisedata[dayfilter] #appluying the day filter
        exercisedata = pd.merge(exercisedata, onerpmdata, on="Name of the exercise", how = "left") #left joining data so we have 1rpms for each exercises
        if randomreps == 1:
            exercisedata["Reps to Perform"] = exercisedata["Reps to Perform"].apply(lambda x: np.random.randint(minreps,maxreps))
            for i in range(len(exercisedata)):
                if exercisedata["Reps to Perform"][i] %2 != 0:
                    exercisedata["Reps to Perform"][i] = ((exercisedata["Reps to Perform"][i]/2) + 0.5) *2 #end me *2 nahi kia to half round ho jayega. end me *2 kia to agar reps 11 aye honge to return me 12 ayenge isntead of 6

            for i in range(len(exercisedata)):
                if exercisedata["Reps to Perform"][i] == 6:
                    exercisedata["Percent 1RM"][i] = 0.85
                elif exercisedata["Reps to Perform"][i] == 7:
                    exercisedata["Percent 1RM"][i] = 0.825
                elif exercisedata["Reps to Perform"][i] == 8:
                    exercisedata["Percent 1RM"][i] = 0.8
                elif exercisedata["Reps to Perform"][i] == 9:
                    exercisedata["Percent 1RM"][i] = 0.775
                elif exercisedata["Reps to Perform"][i] == 10:
                    exercisedata["Percent 1RM"][i] = 0.725 #reducing by 2.5% to get great hypertrophy
                elif exercisedata["Reps to Perform"][i] == 11:
                    exercisedata["Percent 1RM"][i] = 0.7 #reducing by 2.5% to get great hypertrophy
                elif exercisedata["Reps to Perform"][i] == 12:
                    exercisedata["Percent 1RM"][i] = 0.675 #reducing by 2.5% to get great hypertrophy
                elif exercisedata["Reps to Perform"][i] == 14:
                    exercisedata["Percent 1RM"][i] = 0.625 #reducing by 2.5% to get great hypertrophy
                elif exercisedata["Reps to Perform"][i] == 16:
                    exercisedata["Percent 1RM"][i] = 0.575 #reducing by 2.5% to get great hypertrophy
        
        exercisedata["Weight (KG)"] = exercisedata["1rpms (KG)"] * exercisedata["Percent 1RM"] #calculating weight at which we will perform the exercise in kg's and pounds
        exercisedata["Weight (LB)"] = exercisedata["1rpms (LB)"] * exercisedata["Percent 1RM"]
        exercisedata["Percent 1RM"] = exercisedata["Percent 1RM"] * 100 #multiplying 1rpm multiplier by 100 so we see actual % values in final result dataframe
        exercisedata[["Weight (KG)", "Weight (LB)"]] = round(exercisedata[["Weight (KG)", "Weight (LB)"]],2) #rounding to 2 decimal places for better visibility
        exercisedata = exercisedata[["Name of the exercise", "Weak Side", "Tot Timer", "Sets to Perform", "Reps to Perform", "Weight (KG)", #keeping only the required columns in the way we would like to see them
                "Weight (LB)", "1rpms (KG)", "1rpms (LB)", "Instructions"]]
        exercisedata = exercisedata.fillna("-") #filling nulls with - so they dont distract, NaN is distracting
        return exercisedata

    
    #Now, Lets create the weekly calories status calculator!
    #this will check past 2 weeks of our weight and calcualte the difference
    def two_weeks_average_bodyweight(self, dframe): #current calories you can just supply as an integer
        weight_filter = dframe["Category"] == "Weight" #defining filter to have data for bodyweight only from main dataframe
        dframe = dframe[weight_filter] #applying the filter
        max_date = dframe["Date"].max() #getting the max present date in our data
        #we will have to remove timedelta -6 in next step not -7 as we are considering that days date as well. that day to -6 is total 7 days
        sevendays = dframe["Date"].max() - timedelta(days = 6) #we want data for past 7 days so we can take average weight later.
        eighthday = dframe["Date"].max() - timedelta(days = 7) #and this one and below one will be to get data for second week
        fourteenthday = dframe["Date"].max() - timedelta(days = 13)
        currentweek = (dframe["Date"] >= sevendays) & (dframe["Date"] <= max_date) #creating filters for first and second week
        pastweek = (dframe["Date"] >= fourteenthday) & (dframe["Date"] <= eighthday)
        currentweekaverageweight = dframe.loc[currentweek, "Weight (In Kg)"].mean() #calculating mean weight for current and past week
        pastweekaverageweight = dframe.loc[pastweek, "Weight (In Kg)"].mean()
        self.currentaverage = currentweekaverageweight
        self.pastaverage = pastweekaverageweight


    #This fucntion will us eour current calories and find if we are in surplus, deficit or maintainance mode
    def caloriesdata(self, current_calories_that_we_consume_per_day):
    #we have to assign the 2 return values of functions to 2 variables to be able to subtract them later
        currentweekaverageweight = self.currentaverage 
        pastweekaverageweight =  self.pastaverage
        one_pound_calories = 3500 #these are the calories required to gain one pound of weight
        one_pound_grams = 454 #454 grams is one pound
        weight_change_in_kg = currentweekaverageweight - pastweekaverageweight #as the return will be in kg as we are operating on kg
        weight_change_in_grams = weight_change_in_kg * 1000 #as 1000 gram is 1 kg
        calories_gained_or_lost_perweek = (one_pound_calories * weight_change_in_grams) / one_pound_grams
        calories_gained_or_lost_perday = calories_gained_or_lost_perweek / 7
        maintainance_calories = current_calories_that_we_consume_per_day - calories_gained_or_lost_perday
        print(f"7 day average weight is : {currentweekaverageweight}KG")
        print(f"previous week average wegight is : {pastweekaverageweight}KG")
        print(f"our current maintainance calories are: {round(maintainance_calories, 2)}")
        if calories_gained_or_lost_perday > 0: #n\ ke baad space chodoge to vaha behe ayega, directly chipka ke hi type karna padta hai
            print(f"we are currently in a surplus of {round(calories_gained_or_lost_perday, 2)} calories \nOur new bulking calories should be {round(maintainance_calories * 1.10, 2)} \nthey however as of now are: {current_calories_that_we_consume_per_day} \nwhich is a surplus of {round(current_calories_that_we_consume_per_day - (maintainance_calories * 1.1), 2)} calories")
        if calories_gained_or_lost_perday <= 0:
            print(f"we are currently in a deficit of {round(calories_gained_or_lost_perday, 2)} calories \nOur new cutting calories should be {round(maintainance_calories * 0.8, 2)} \nthey however as of now are: {current_calories_that_we_consume_per_day} \nwhich is a surplus / deficit of {round(current_calories_that_we_consume_per_day - (maintainance_calories * 0.8), 2)}")
        print("")
        if currentlycutting == 1:
            cutting_calories = round(maintainance_calories * 0.75,2)
            cutting_protein = round((currentweekaverageweight*2.2) * 1.2,2) #as 1.6 grams per pound of bodyweight
            cutting_protein_calories = round(cutting_protein * 4, 2)
            cutting_fat_calories = round(cutting_calories * 0.2, 2) #as 25% should coemf rom fat
            cutting_fat = round(cutting_fat_calories / 9, 2)
            cutting_carbs_calories = round(cutting_calories - (cutting_protein_calories + cutting_fat_calories),2)
            cutting_carbs = round((cutting_carbs_calories / 4), 2)
            print(f"cutting calories (75% of maintainance) are equal to {cutting_calories} calories")
            print(f"go for 1.2 grams of per pound of protein and you should take: {cutting_protein} grams of protein and that would be {cutting_protein_calories} calories")
            print(f"go for 25% calories from fat and that should be : {cutting_fat} grams of fat and that would be {cutting_fat_calories} calories")
            print(f"rest of the calories should come from carbs and that should be : {cutting_carbs} grams of carbs and that would be {cutting_carbs_calories} calories")
    
    #Now we take measurements every sunday and we have our target measurements, we want a fucntion that cmpares our current measurements to our target measurements and figures out which is the most lacking body part, hence labelling it as week body part
    def weakbodypart(self,dataset):
        body_measurement_filt = (dataset["Category"] == "Body Measurement") #filtering out measurements part for max available date within itself
        measurement_dataset = dataset.loc[body_measurement_filt]
        measurement_max_date = (measurement_dataset["Date"] == measurement_dataset["Date"].max())
        measurement_dataset = measurement_dataset.loc[measurement_max_date, ["Muscle", "Measurement (in inches)"]]

        target_measurement_filt = (dataset["Category"] == "Target Measurement")  #filtering out targets part for max available date within itself
        target_dataset = dataset.loc[target_measurement_filt]
        target_max_date = (target_dataset["Date"] == target_dataset["Date"].max())
        target_dataset = target_dataset.loc[target_max_date, ["Muscle", "Measurement (in inches)"]]

        measurement_set = pd.merge(target_dataset, measurement_dataset, how = "inner", on = "Muscle")
        #humne left side target rakhah hai (first in syntax) and right side measuremnts. to suffixes me x will be targets and y will be measurements
        #multiply by -1 to change positive signs to negatives
        measurement_set["Percentage Difference from goal"] = round(((((measurement_set["Measurement (in inches)_y"] - measurement_set["Measurement (in inches)_x"])* -1) / measurement_set["Measurement (in inches)_x"]) * 100) , 2)
        measurement_set = measurement_set.sort_values(by = "Percentage Difference from goal", ascending = False)
        measurement_set = measurement_set.rename(columns = {"Measurement (in inches)_x" : "Target Measurements", "Measurement (in inches)_y" : "Current Measurements"})
        return measurement_set



In [29]:
#creating the class object
fa = fitnessanalytics()

In [38]:
#calling our class with necessary arguments
dategreaterthan = "2022-11-20"
currentlycutting = 0 #1 means yes 0 means no. cut pe all 1rpms will be reduced by 30%. as we will be fasted.
randomreps = 1
week = 3
dayoftheweek = input("Enter Day of the week:")
dayoftheweek = int(dayoftheweek)

fa.OneRPM(onerpmdata, dategreaterthan)
fa.workout(exercisedata, week = week, dayoftheweek = dayoftheweek, randomreps = randomreps)

,Name of the exercise,Weak Side,Tot Timer,Sets to Perform,Reps to Perform,Weight (KG),Weight (LB),1rpms (KG),1rpms (LB),Instructions
0,Machine Shoulder Press,-,-,2,6,50.60,111.32,59.53,130.97,-
1,Rows Machine,-,-,3,8,89.18,196.18,111.47,245.23,-
2,Decline Dip Press,-,-,3,12,162.00,356.40,240.00,528.00,-
3,Elevated Rope Rows,Right,-,3,12,55.57,122.26,82.33,181.13,-
4,Incline Dip Press,-,-,3,8,91.20,200.64,114.00,250.80,5 no pe rakhna hai rope
5,Wide Grip Lat Pulldown,Right,-,3,8,45.92,101.02,57.40,126.28,Be straight don’t go back like lat pulldown
6,Inner Dip Press,-,-,3,8,96.00,211.20,120.00,264.00,Keep on steel ot cushion
7,Straight Arm Lat Pulldown,-,-,3,8,30.80,67.76,38.50,84.70,no 5 pe rakho


In [16]:
#now, getting our calories related details
fa.two_weeks_average_bodyweight(onerpmdata)
fa.caloriesdata(2500)

7 day average weight is : 63.557142857142864KG
previous week average wegight is : 63.300000000000004KG
our current maintainance calories are: 2216.8
we are currently in a surplus of 283.2 calories 
Our new bulking calories should be 2438.48 
they however as of now are: 2500 
which is a surplus of 61.52 calories



In [32]:
#figuring out week body parts
fa.weakbodypart(onerpmdata)

,Muscle,Target Measurements,Current Measurements,Percentage Difference from goal
10,Left Calf,15.31,14.1,7.90
9,Right Calf,15.31,14.4,5.94
11,Neck,15.31,14.5,5.29
5,Bicep (right),15.31,14.8,3.33
6,Bicep (left),15.31,14.8,3.33
7,Right Thigh,22.58,22.0,2.57
8,Left Thigh,22.58,22.0,2.57
3,Lower Chest,38.80,37.9,2.32
12,Right Forearm,11.25,11.1,1.33
4,Shoulders,45.30,45.0,0.66
